# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=8

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==8]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm8', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm8/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-20 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]     [,2]          [,3]         [,4]
sigmas -0.494071936 2.601265 -11.969150733  56.69156104
       -0.487744053 2.559450 -11.799912644  56.57978940
       -0.481455961 2.518027 -11.628820322  56.45237282
       -0.475207162 2.477015 -11.456008353  56.30964612
       -0.468997168 2.436434 -11.281608558  56.15194116
       -0.462825500 2.396301 -11.105750009  55.97958682
       -0.456691688 2.356633 -10.928559053  55.79290897
       -0.450595271 2.317445 -10.750159336  55.59223037
       -0.444535794 2.278752 -10.570671820  55.37787069
       -0.438512814 2.240568 -10.390214812  55.15014642
       -0.432525893 2.202908 -10.208903987  54.90937090
       -0.426574603 2.165783 -10.026852415  54.65585423
       -0.420658520 2.129205  -9.844170586  54.38990332
       -0.414777232 2.093185  -9.660966436  54.11182178
       -0.408930331 2.057734  -9.477345379  53.82191002
       -0.40311

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]        [,3]        [,4]
sigmas -0.532840654 1.4629276 -3.21670354 -0.12017786
       -0.526376180 1.4508811 -3.21992442  0.02486875
       -0.519953227 1.4388280 -3.22273118  0.17103879
       -0.513571265 1.4267688 -3.22511017  0.31827285
       -0.507229774 1.4147042 -3.22704805  0.46651109
       -0.500928245 1.4026349 -3.22853187  0.61569331
       -0.494666176 1.3905616 -3.22954904  0.76575897
       -0.488443078 1.3784855 -3.23008733  0.91664720
       -0.482258466 1.3664073 -3.23013489  1.06829683
       -0.476111869 1.3543283 -3.22968025  1.22064646
       -0.470002822 1.3422496 -3.22871233  1.37363442
       -0.463930869 1.3301724 -3.22722043  1.52719887
       -0.457895562 1.3180981 -3.22519426  1.68127779
       -0.451896462 1.3060280 -3.22262393  1.83580903
       -0.445933137 1.2939637 -3.21949994  1.99073031
       -0.440005162 1.2819068 -3.21581322  2.14597930
       -0.434112120 1.2698588 -3.21155509  2.301

        0.146619328 0.5760300  1.14890180  7.85528676
        0.149901332 0.5806778  1.17766901  7.73017782
        0.153172600 0.5854316  1.20581624  7.60282427
        0.156433201 0.5902884  1.23333300  7.47324500
        0.159683205 0.5952454  1.26020897  7.34145916
        0.162922682 0.6002998  1.28643392  7.20748604
        0.166151697 0.6054485  1.31199780  7.07134517
        0.169370320 0.6106887  1.33689065  6.93305624
        0.172578616 0.6160172  1.36110268  6.79263914
        0.175776653 0.6214309  1.38462422  6.65011392
        0.178964494 0.6269269  1.40744575  6.50550082
        0.182142205 0.6325019  1.42955788  6.35882025
        0.185309851 0.6381527  1.45095138  6.21009278
        0.188467494 0.6438762  1.47161715  6.05933915
        0.191615198 0.6496690  1.49154624  5.90658025
        0.194753024 0.6555279  1.51072983  5.75183715
        0.197881036 0.6614496  1.52915929  5.59513104
        0.200999294 0.6674306  1.54682609  5.43648329
        0.204107858 0.673467

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_14739/1308013083.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]        [,3]       [,4]
sigmas -0.334722882 1.0427051 -2.43851635 -0.5172522
       -0.330152719 1.0349587 -2.44559268 -0.3099032
       -0.325603346 1.0271781 -2.45188806 -0.1035945
       -0.321074577 1.0193663 -2.45740527  0.1016556
       -0.316566225 1.0115259 -2.46214716  0.3058280
       -0.312078107 1.0036595 -2.46611661  0.5089034
       -0.307610043 0.9957699 -2.46931657  0.7108617
       -0.303161853 0.9878598 -2.47175005  0.9116823
       -0.298733362 0.9799318 -2.47342014  1.1113442
       -0.294324396 0.9719886 -2.47432998  1.3098259
       -0.289934784 0.9640329 -2.47448279  1.5071051
       -0.285564356 0.9560674 -2.47388190  1.7031592
       -0.281212946 0.9480946 -2.47253067  1.8979652
       -0.276880389 0.9401172 -2.47043261  2.0914995
       -0.272566522 0.9321379 -2.46759127  2.2837379
       -0.268271184 0.9241592 -2.46401033  2.4746559
       -0.263994217 0.9161838 -2.45969356  2.6642285
       -0.25

        0.193699801 0.6181688  1.56095131  8.1573727
        0.196396595 0.6234210  1.58562432  8.0518917
        0.199086136 0.6287496  1.60985954  7.9449951
        0.201768462 0.6341529  1.63365203  7.8367004
        0.204443613 0.6396293  1.65699695  7.7270252
        0.207111626 0.6451771  1.67988952  7.6159871
        0.209772541 0.6507946  1.70232505  7.5036032
        0.212426393 0.6564801  1.72429892  7.3898908
        0.215073221 0.6622320  1.74580658  7.2748670
        0.217713062 0.6680484  1.76684357  7.1585486
        0.220345953 0.6739276  1.78740549  7.0409527
        0.222971929 0.6798679  1.80748805  6.9220959
        0.225591028 0.6858676  1.82708700  6.8019949
        0.228203285 0.6919247  1.84619817  6.6806663
        0.230808736 0.6980376  1.86481749  6.5581266
        0.233407417 0.7042044  1.88294094  6.4343922
        0.235999361 0.7104233  1.90056460  6.3094794
        0.238584605 0.7166924  1.91768460  6.1834045
        0.241163183 0.7230099  1.93429717  6.0

In [2]:
print(error_i)

[1, 2, 3, 6, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 36, 37, 38, 43, 55, 107, 224]
